In [1]:
from ipywidgets import HTML
from ipyleaflet import Map,Marker,AwesomeIcon,Popup,AntPath,LayerGroup,\
                       FullScreenControl,LayersControl,ScaleControl,MeasureControl
import pandas as pd
import numpy as np

## 读取数据

In [2]:
cdf = pd.read_csv('./social_most_checkins.csv',parse_dates=['created_at'])

In [3]:
cdf

,user_id,venue_id,created_at,latitude_venue,longitude_venue,latitude_home,longitude_home
0,48,5222,2012-01-31 04:01:42,37.62,-122.40,37.44,-122.20
1,61,1619,2012-01-28 06:45:05,26.16,-80.10,40.72,-74.00
2,101,52313,2011-12-09 04:21:35,37.78,-122.44,37.78,-122.44
3,101,52313,2011-12-09 04:21:35,37.78,-122.44,37.78,-122.44
4,485,183188,2011-12-26 07:41:41,50.06,8.57,37.78,-122.44
5,485,5222,2011-12-26 21:26:02,37.62,-122.40,37.78,-122.44
6,485,7620,2012-01-23 21:43:23,33.94,-118.40,37.78,-122.44
7,485,2966,2012-03-16 19:38:31,30.27,-97.75,37.78,-122.44
8,485,182463,2012-03-17 14:27:02,30.27,-97.75,37.78,-122.44
9,485,182463,2012-03-18 16:14:32,30.27,-97.75,37.78,-122.44


In [4]:
icon_start = AwesomeIcon(
    name='flag',
    marker_color='red',
    icon_color='black',
    spin=False
)

icon_end = AwesomeIcon(
    name='flag',
    marker_color='green',
    icon_color='black',
    spin=False
)

In [5]:
center = (0,0)
m = Map(center=center, zoom=1.5)

In [6]:
m.add_control(FullScreenControl())
control = LayersControl(position='topright')
m.add_control(control)
m.add_control(ScaleControl(position='bottomleft',imperial=False))
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    completed_color = 'red',
    primary_length_unit = 'kilometers',
    primary_area_unit = 'sqmiles'
)
m.add_control(measure)

In [7]:
for idx, value in cdf.groupby('user_id'):
    icon_list = []
    icon_location = []
    icon_dict = {}
    icon_num_dict = {}
    i = 0
    j = 0
    for v in value.venue_id:
        
        if i==0:
            marker = Marker(draggable=False)
            marker.icon = icon_start
            icon_list.append(marker)

        if v not in icon_dict.keys():
            if i!=0:
                marker = Marker(draggable=False)
                icon_list.append(marker)
            icon_dict[v] = str(value.iloc[i,2])
            icon_num_dict[v] = j
            j+=1
            icon_location.append(list(value.iloc[i,3:5].values))

        else:
            icon_dict[v] = icon_dict[v] + '<br>' + str(value.iloc[i,2])
        
        if j==len(np.unique(value.venue_id)):
            icon = icon_list[icon_num_dict[v]]
            icon.icon = icon_end
            icon_list[icon_num_dict[v]] = icon
        
        i+=1
    
    popup = [info for _,info in icon_dict.items()]
    user = ['id:' + str(idx)+' '+ '<br>' + 'venue:' + str(i) for i in range(len(icon_dict))]
    
    i = 0
    new_icon_list = []
    for icon in icon_list:
        
        icon.location = icon_location[i]
        
        message=HTML()
        message.value = popup[i]
        message.placeholder = user[i]
        message.description = user[i]
        icon.popup = message
        
        new_icon_list.append(icon)
        i = i + 1
    
    ant_path = AntPath(dash_array=[1, 10],delay=1000)
    ant_path.locations = icon_location
    new_icon_list.append(ant_path)
    
    layer_group = LayerGroup(layers=new_icon_list, name='id-'+str(idx))
    m.add_layer(layer_group)

In [8]:
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…